# MI primer random forest con scikit-learn

## Conociendo nuestro dataset

Utilizaremos el [Pima indians diabetes](https://www.kaggle.com/datasets/kumargh/pimaindiansdiabetescsv)

El conjunto de datos contiene información médica de mujeres Pima Indian de Arizona, Estados Unidos, que participaron en un estudio de la diabetes en la década de 1980.

El conjunto de datos consta de 768 instancias y 9 atributos, incluyendo el número de veces que una mujer ha estado embarazada, su edad, presión arterial diastólica, índice de masa corporal, concentración de glucosa en plasma y la presencia o ausencia de diabetes en la prueba.

### Atributos

1. preg: Número de veces embarazada

2. plas: Concentración de glucosa en plasma a las 2 horas en una prueba de tolerancia a la glucosa oral

3. pres: Presión arterial diastólica (mm Hg)

4. skin: Grosor del pliegue cutáneo tricipital (mm)

5. test: Concentración de insulina en suero a las 2 horas (mu U/ml)

6. mass: Índice de masa corporal (peso en kg/(altura en m)^2)

7. pedi: Función de diabetes basada en antecedentes familiares

8. age: Edad (años)

9. class: Variable de clase (1: positivo para diabetes, 0: negativo para diabetes en la prueba)

In [ ]:
# Importando las librerias necesarias
import matplotlib.pyplot as plt

# Datos faltantes
import missingno as miss
import numpy as np
import pandas as pd
import seaborn as sns

# Modelo de Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

# Metricas de nuestro modelo
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Modelo y procesamiento de datos
# Optimizacion de parametros de nuestro modelo
# Validacion Cruzada
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    cross_val_score,
    train_test_split,
)

sns.set(style="whitegrid", context='notebook', palette="Spectral")
# sns.color_palette("blend:#7AB,#EDA", as_cmap=True)

In [ ]:
# Cargamos el dataset a utilizar
df = pd.read_csv("../data/pima-indians-diabetes.csv", sep=',')

In [ ]:
# Visualizancion aleatoria de 10 registros
df.sample(10)

In [ ]:
# Renombrando columnas
columns_names = np.array(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'Class'])

df.columns = columns_names

In [ ]:
# Verificacion de los datos
df.head()

Todos los datos son de tipo numerico

Nuestro target es Class

In [ ]:
# Analizamos el shape
print(df.shape)

In [ ]:
# Visualizamos los tipos de datos
df.info()

# Data pre-processing


# Exploracíon de datos

In [ ]:
miss.matrix(df);

no se encuentran valores nulos

In [ ]:
# Manejo de datos nulos
df.isna().sum()

Variables con datos perdidos

In [ ]:
df.iloc[:, 1:6].replace(to_replace=[0], value=np.nan).isna().sum().reset_index(name = 'missing_values').rename(columns={"index": "variable"}).assign( percentage = lambda df_reset: df_reset.missing_values / len(df) * 100)

Porporcion de la variable objetivo

In [ ]:
plt.figure(figsize=(8,8))

labels, counts = np.unique(df['Class'], return_counts=True)
plt.pie(counts, labels=labels, autopct='%1.1f%%')
plt.legend({'Diabetes Negativo', 'Diabetes positivo'})
plt.title("Proporcion de diabetes")
plt.show()

# 
print(df['Class'].value_counts())

## Entrenamiento de random forest con scikit-learn

In [ ]:
X = df.drop('Class', axis=1)
y = df['Class']

In [ ]:
X

In [ ]:
y

In [ ]:
# 30% para test y 70% para train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
# Instanciar nuestro modelo de random forest
rfc_model = RandomForestClassifier(random_state=42)

In [ ]:
# Definir los hiperparametros y sus posibles valores
param_grid = {
    "n_estimators": [5, 10, 25, 50],
    "max_depth": [5, 10, 15],
    "criterion": ["gini", "entropy", "log_loss"],
    "min_samples_split": [2, 4, 6],
    "min_samples_leaf": [1, 2, 4]
}

In [ ]:
# Crear el objeto GridSearchCv
grid_search = GridSearchCV(estimator=rfc_model, param_grid=param_grid, cv=5, scoring='accuracy')

In [ ]:
# Ajustar el modelo con Grid
grid_search.fit(X_train, y_train)

In [ ]:
for params, mean_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score']):
    print(f"Parametros: {params}\n")
    print("="*70)
    print(f"Mean Score: {mean_score}")

Cual es la mejor optimizacion de nuestro modelo

In [ ]:
print(f"Mejores hiperParametros encontrados: {grid_search.best_params_}")
print("-"*50)
print(f"Mejor puntuacion de validacion cruzada: {grid_search.best_score_}\n")

In [ ]:
# Obtener el mmodelo con el mejor rendimiento

best_rfc = grid_search.best_estimator_

In [ ]:
# Calculo de las predicciones en Train y Test
y_train_pred = best_rfc.predict(X_train)
# Test
y_test_pred = best_rfc.predict(X_test)

# Evaluacion de nuestro modelo

In [ ]:
train_accuracy = accuracy_score(y_train, y_train_pred)
# Prediccion test
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Train accuracy: ", train_accuracy)
print("Test accuracy: ", test_accuracy)

In [ ]:
# reporte de classification
print(classification_report(y_test, y_test_pred))

El modelo de Random Forest, con parámetros optimizados, alcanzó una **precisión general** del *74%* y un **promedio ponderado** del *71%*

El F1-score para la **clase "0"** fue de *80%*, indicando una buena capacidad para identificar casos negativos. Sin embargo, el F1-score para la **clase "1"** fue de *62%*, mostrando un rendimiento ligeramente inferior en la detección de casos positivos.

En general, el modelo demostró un rendimiento satisfactorio.

In [ ]:
cm = confusion_matrix(y_test, y_test_pred, labels=best_rfc.classes_)

sns.heatmap(cm, annot=True, fmt="d", cmap='gray', square=True, cbar=False)
plt.xlabel("True Label")
plt.ylabel("Predicted Label")
plt.show()

# Análisis de las importancias de los features

In [ ]:
feature_scores_diabetes_df = pd.DataFrame({
    "Feature": list(X.columns),
    "importance": best_rfc.feature_importances_
}).sort_values("importance", ascending=False)

In [ ]:
feature_scores_diabetes_df

In [ ]:
sns.barplot(data=feature_scores_diabetes_df, y='Feature', x='importance', hue='importance', palette="Set2", legend=False)
plt.title('Feature Importance')
plt.yticks(rotation=45)
plt.show()

In [ ]:
feature_scores_diabetes = pd.DataFrame(pd.Series(best_rfc.feature_importances_, index=X_train.columns).sort_values(ascending=False)).T
plt.figure(figsize=(8,6))
sns.barplot(data=feature_scores_diabetes)

for index, value in enumerate(feature_scores_diabetes.values.flatten()):
    plt.annotate(f'{value:.2f}', xy=(index, value), ha='center', va='bottom')


plt.title("Factores clave en la predicción de diabetes positivo en pima indians")
plt.show()
feature_scores_diabetes_df

Un análisis de las importancias de las características revela que las características más influyentes para la predicción del modelo son:

- plas: Con una importancia de *0.253495*, se destaca como la característica más importante. Esto indica que la concentración de glucosa en plasma sanguíneo tiene un impacto significativo en la predicción de la diabetes.

- mass: El índice de masa corporal (IMC) es la segunda característica más relevante, con una importancia de *0.189541*. Esto sugiere que el peso relativo a la altura también juega un papel crucial en la predicción de la diabetes.

- age: La edad se posiciona como la tercera característica más importante, con una importancia de *0.152137*. Esto indica que la edad de la paciente también contribuye significativamente a la predicción de la enfermedad.

- pedi: El valor de pedigree diabetes function (pedi) tiene una importancia de *0.116485*. Esto sugiere que el historial familiar de diabetes puede tener un impacto en la predicción de la enfermedad.

Las características restantes, como **test, preg, pres y skin** tienen importancias relativamente más bajas en comparación con las anteriores, pero aún contribuyen al proceso de predicción.

Estos resultados resaltan la importancia de características clave como la concentración de glucosa, el índice de masa corporal, la edad y el historial familiar en la predicción de la diabetes en el conjunto de datos.

# Validacion Cruzada

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k =5
cv = KFold(n_splits=k, shuffle=True, random_state=42)

scores = cross_val_score(best_rfc, X_train, y_train, cv=cv, scoring='accuracy')
print("")
print("Accuracy score for each fold: ", scores * 100)
print("Mean accuracy: ", scores.mean() * 100)
print("Stadard deviation: ", scores.std() * 100)

Los resultados muestran que el modelo tiene una **precisión promedio** del *74.8113%* en la validación cruzada, con una **desviación estándar** de *2.1526%*, lo que indica que las puntuaciones de precisión están relativamente cerca de la media y son consistentes entre los pliegues.

Esto sugiere que el modelo tiene un rendimiento razonablemente estable y generaliza bien en los diferentes pliegues de la validación cruzada.